In [11]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score

# import some data to play with
dfstream = pd.read_csv('base.csv')
X = dfstream.drop(["dileso", "x", "y", "cruce_osm"], axis = 1)
y = dfstream["dileso"]

In [3]:
dfstream.head()

,Id,anio,mes,dia,dia_sem,clave_mun,mun,herido,ibaen_atro,num_mes,rango_hora,tipo_accidente,cruce_setrans,cruce_osm,rango_edad,x,y,tipo_usuario,sexo,dileso
0,2015-01-01-620850-SISCAV-08:00 A 09:59,2015,Enero,1,jueves,98.0,San Pedro Tlaquepaque,NaN,Automovil,1,8:00 a 8:59,Contra Objeto Fijo,TLAQUEPAQUELAZARO CARDENAS CALZ,NaN,18-27,NaN,NaN,Conductor,Hombre,1
1,2015-01-01-620850-SISCAV-08:00 A 09:59,2015,Enero,1,jueves,98.0,San Pedro Tlaquepaque,Lesionado,Automovil,1,8:00 a 8:59,Contra Objeto Fijo,TLAQUEPAQUELAZARO CARDENAS CALZ,NaN,No disponible,NaN,NaN,Pasajero,Mujer,0
2,2015-01-17-633262-SISCAV-06:00 A 07:59,2015,Enero,17,sábado,39.0,Guadalajara,Lesionado,Automovil,1,6:00 a 6:59,Atropellamiento,GARIBALDICOLONIA CENTRO,NaN,28-37,NaN,NaN,Conductor,Hombre,0
3,2015-01-17-633262-SISCAV-06:00 A 07:59,2015,Enero,17,sábado,39.0,Guadalajara,Lesionado,Automovil,1,6:00 a 6:59,Atropellamiento,GARIBALDICOLONIA CENTRO,NaN,No disponible,NaN,NaN,Peatón,Hombre,0
4,2015-01-18-626486-SISCAV-16:00 A 17:59,2015,Enero,18,domingo,39.0,Guadalajara,Lesionado,Motocicleta,1,16:00 a 16:59,Contra Objeto Fijo,ESTEBAN ALATORREABRAHAM GONZALEZ,NaN,48-59,NaN,NaN,Conductor,Hombre,0


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
from sklearn.preprocessing import LabelEncoder

string_columns = X.select_dtypes(include=['object']).columns
# Apply LabelEncoder to the string columns
for column in string_columns:
    X[column] = LabelEncoder().fit_transform(X[column])
    

# Check the data type of the series
data_type = y.dtype

# Apply LabelEncoder to the series if it is of object type
if data_type == 'object':
    y = LabelEncoder().fit_transform(y)
    
X = X.fillna(X.mode().iloc[0])
    
    
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
    
polynomial_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly = polynomial_features.fit_transform(X)

model_rl = LinearRegression()

model_rl.fit(X, y)

model_rl.score(X_test, y_test)

y_pred = model_rl.predict(X_train)

df_test = pd.read_csv("base.csv")

df_test = df_test.drop(["dileso", "x", "y", "cruce_osm"], axis = 1)


string_columns = df_test.select_dtypes(include=['object']).columns

# Apply LabelEncoder to the string columns
for column in string_columns:
    df_test[column] = LabelEncoder().fit_transform(df_test[column])
    
df_test = df_test.fillna(df_test.mode().iloc[0])

y_hat = model_rl.predict(df_test)

def transform_data(y_hat):
    transformed_data = []  # Initialize an empty list to store the transformed values
    
    for value in y_hat:
        if value < 0.5:
            transformed_data.append(0)
        else:
            transformed_data.append(1)
    
    return transformed_data

transformed_values = transform_data(y_hat)

df_submission = pd.DataFrame(df_test["Id"])
df_submission ["dileso"] = transformed_values

In [13]:
import pickle

filename = 'model.sav'
pickle.dump(df_submission, open(filename, 'wb'))